In [45]:
import os
!pip install -U langchain-community cohere together

import cohere

from langchain_community.embeddings import CohereEmbeddings
from langchain_community.llms import Together

os.environ["COHERE_API_KEY"] = "RniGQNO1HjzrZN4nhUnb0ILAjhPFapH1MqKuaIwW"
os.environ["TOGETHER_API_KEY"] = "279ab1fc40b131bc2ecead5df84c52c8ac69e3b255795854e4ef44cf5a3ae785"



Install libraries

In [14]:
!pip install -q youtube-transcript-api langchain-community langchain-cohere langchain-together \
             faiss-cpu tiktoken python-dotenv


In [22]:
from youtube_transcript_api import YouTubeTranscriptApi, TranscriptsDisabled
from langchain.text_splitter import CharacterTextSplitter,RecursiveCharacterTextSplitter
from langchain.embeddings import CohereEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate
from langchain.llms import Together

INDEXING


In [84]:
from youtube_transcript_api import YouTubeTranscriptApi, TranscriptsDisabled

video_id ="Gfr50f6ZBvo"

try:
  transcript_list= YouTubeTranscriptApi.get_transcript(video_id, languages=["en"])

  transcript="".join(chunk["text"] for chunk in transcript_list)
  print(transcript)

except TranscriptsDisabled:
  print("Transcripts are disabled for this video.")

the following is a conversation withdemus hasabisceo and co-founder of deepminda company that has published and buildssome of the most incredible artificialintelligence systems in the history ofcomputing including alfred zero thatlearnedall by itself to play the game of goldbetter than any human in the world andalpha fold two that solved proteinfoldingboth tasks considered nearly impossiblefor a very long timedemus is widely considered to be one ofthe most brilliant and impactful humansin the history of artificialintelligence and science and engineeringin generalthis was truly an honor and a pleasurefor me to finally sit down with him forthis conversation and i'm sure we willtalk many times again in the futurethis is the lex friedman podcast tosupport it please check out our sponsorsin the description and now dear friendshere's demishassabislet's start with a bit of a personalquestionam i an ai program you wrote tointerview people until i get good enoughto interview youwell i'll be imp

TEXT SPLITTER

In [85]:
splitter= RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=300)
chunks=splitter.create_documents([transcript])
len(chunks)

186

EMBEDDING & STORE IN VECTOR STORE

In [86]:
embeddings=CohereEmbeddings (model="embed-english-v3.0",user_agent="langchain")
vector_store=FAISS.from_documents(chunks,embeddings)

In [87]:
vector_store.index_to_docstore_id

{0: '84de7b86-c660-44bb-9024-a90cb3b2cef9',
 1: '808a1172-ab92-437f-bad1-4680db566f9e',
 2: '3a88a1f7-6cf1-40ca-accb-4a1618a9b297',
 3: 'ddaee76e-049d-4013-8b09-b707a7268412',
 4: 'b385524e-9721-438c-a958-32c8f6baa46c',
 5: 'e2d7183a-0a36-42aa-9500-2908d3d8f432',
 6: '2e828dc5-3a51-472b-b425-5e86e9760aac',
 7: '660f43a5-e53f-444f-947e-31e3d64723e0',
 8: '3c4b2ffb-fa41-4b2f-8c41-d3a749cc19be',
 9: 'ff321d58-f39b-4d6c-8c67-e38db2fe59b8',
 10: '760c13e7-5879-4697-8373-02bc934eb9cb',
 11: '63fe2447-8e63-4924-9752-92272391303d',
 12: '479adb71-8bd9-49d7-a5fb-eff7225c3aed',
 13: 'fbf5f7cd-9385-4892-b0c3-a4fe676b7c08',
 14: 'f04fd0ad-afb1-4caf-8418-ca99f6619983',
 15: 'c4ef0802-ebb7-4cd1-91bd-af23a9cd37ca',
 16: 'b7a38ed0-2744-465c-b07c-33fb7d1ba1a1',
 17: '7ea281e5-3726-4759-835d-a6b5f05bab6c',
 18: '408d8680-3f9d-4c75-9375-e8b0f0f55c48',
 19: 'b3c54385-56e1-4d2e-b443-6e6c038e8169',
 20: 'e62e48d8-d518-44ac-b126-b90bc452c8d3',
 21: '44900406-6c94-4696-b724-aeec8fc6e4b1',
 22: '3887da05-3870-

see chunk using id

In [88]:
vector_store.get_by_ids(['eccf43e1-7916-4aff-b9d7-51429f6f7857'])

[]

In [107]:
retriever= vector_store.as_retriever(search_type="similarity", search_kwargs={"k":4})

In [92]:
retriever.invoke('what is deepmind')

[Document(id='6b6ffa1e-6c5d-41f8-8c49-f98ae7b07d81', metadata={}, page_content="theartifact first and then you can studyhow how and pull it apart and how itworks this is tough to uhask you this question because youprobably will say it's everything butlet's let's try let's try to think tothis because you're in a veryinteresting position where deepmind isthe place of some of the most uhbrilliant ideas in the history of ai butit's also a place of brilliantengineeringso how much of solving intelligence thisbig goal for deepmind how much of it isscience how much is engineering so howmuch is the algorithms how much is thedata how much is thehardware compute infrastructure how muchis it the software computerinfrastructure yeah um what else isthere how much is the humaninfrastructureand like just the humans interact incertain kinds of ways in all the spaceof all those ideas how much does maybelike philosophy how much what's the keyif umuhif if you were to sort of look back likeif we go forward

AUGMENTATION

In [93]:
os.environ["TOGETHER_API_KEY"] = "279ab1fc40b131bc2ecead5df84c52c8ac69e3b255795854e4ef44cf5a3ae785"
from dotenv import load_dotenv
load_dotenv()
from langchain_together import Together

llm = Together(
    model="mistralai/Mixtral-8x7B-Instruct-v0.1",
    temperature=0.7,
    max_tokens=512,
)


In [94]:
prompt = PromptTemplate(
    input_variables=['context', 'question'],
    template="""
You are a helpful AI assistant.
Answer ONLY from the provided transcript context.
If the context is insuffiecient, just say you don't know.

Context:
{context}

Question:
{question}

Helpful Answer:
"""
)

In [95]:
question="os the topic of aliens discussed ijn nthe video?If yes, what was discussed?"
retrieved_docs= retriever.invoke(question)

In [96]:
content_text="\n\n".join(doc.page_content for doc in retrieved_docs)

In [97]:
final_prompt=prompt.invoke({"context":content_text, "question":question})

GENERATION

In [98]:
answer= llm.invoke(final_prompt)
print(answer)


Yes, the topic of aliens was discussed in the video. The speaker mentioned that they think the most likely scenario is that we are alone, given the evidence we have so far. They also mentioned projects like the SETI program and the dawning of the space age, where telescopes and radiotelescopes have been used to detect signals. The speaker also discussed the simulation hypothesis, suggesting that what we're seeing might not be reality, but a hologram projected by aliens or something more fundamentally different that we're too dumb to understand. They also mentioned the possibility that diseases and big challenges we face could be methods of communication from these aliens. The speaker expressed their hope that AI can help us solve these challenges, find aliens if they exist, and influence the values of the creators of AI systems. They also mentioned the argument that we haven't done an exhaustive search yet, but they don't really buy this argument. They also mentioned the 'Star Trek rul

In [99]:
from langchain_core.runnables import RunnablePassthrough,RunnableParallel,RunnableLambda
from langchain_core.output_parsers import StrOutputParser

In [100]:
def format_docs(docs):
  context_text= "\n\n".join(doc.page_content for doc in docs)
  return context_text

In [101]:
parallel_chain= RunnableParallel(
    context= retriever | RunnableLambda(format_docs),
    question=RunnablePassthrough()
)

In [ ]:
parallel_chain.invoke("who is demis")

In [103]:
parser=StrOutputParser()

In [104]:
main_chain=parallel_chain | prompt | llm | parser

In [116]:
main_chain.invoke("summarise")

"The speaker is discussing the complexity of simulating materials and their properties, which is governed by electron behavior. They've generated a lot of data from molecular dynamics simulations, which they use to train their AI system. The speaker compares the process to appreciating complex topics, like music or understanding proteins, which are specified by their genetic sequence and fold into a 3D structure to determine their function. The speaker also mentions different levels of AI intelligence, from interpolation (generating an average-looking cat from millions of cat pictures) to extrapolation (coming up with new strategies in games like Go) to true innovation (inventing a new game like chess)."

In [121]:
!pip install flask_cors
from flask import Flask, request, jsonify
from flask_cors import CORS

app = Flask(__name__)
CORS(app)

@app.route("/chat", methods=["POST"])
def chat():
    data = request.json
    query = data.get("query")
    video_id = data.get("video_id")

    # Run your RAG logic here
    result = run_rag_pipeline(query, video_id)  # <- your function
    return jsonify({"answer": result})

app.run(port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


In [122]:
!pip install flask-ngrok
from flask_ngrok import run_with_ngrok
run_with_ngrok(app)
